# Sample commodity yield forescast dashboard

This notebook will generate synthetic but realistic crop yields and weather data to set up a mock dashboard for commodity yield forecasting using python only

# 1️⃣ Imports and base configuration

In [84]:
import argparse
import os  # 👈 added import for os.path.join
from pathlib import Path
import numpy as np
import pandas as pd
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ----------------------------
# Global settings and reproducibility
# ----------------------------
np.random.seed(42)
OUTDIR = "output"  # keep as string for os.path.join compatibility
os.makedirs(os.path.join(".", OUTDIR), exist_ok=True)  # 👈 replaced Path.mkdir


# ----------------------------
# Define hierarchy of geography
# ----------------------------
# Australia’s pea-growing example:
# - 2 States
# - 5 Regions (3 in WA, 2 in SA)
# - 25 Municipalities total
STATES = {
    "Western Australia": {
        "regions": {
            "Great Southern": ["Albany", "Denmark", "Plantagenet", "Cranbrook", "Broomehill-Tambellup"],
            "Wheatbelt": ["Northam", "Toodyay", "York", "Beverley", "Quairading"],
            "South Coast": ["Esperance", "Ravensthorpe", "Jerramungup", "Gnowangerup", "Kojonup"],
        }
    },
    "South Australia": {
        "regions": {
            "Yorke Peninsula": ["Copper Coast", "Yorke Peninsula", "Barunga West", "Wakefield", "Wallaroo"],
            "Mid North": ["Clare and Gilbert Valleys", "Goyder", "Light", "Mallala", "Peterborough"],
        }
    },
}


# 2️⃣ Build geography and helper functions

In [85]:
def build_geo():
    """Create a table of all state/region/municipality combinations."""
    rows = []
    for st, st_data in STATES.items():
        for reg, munis in st_data["regions"].items():
            for m in munis:
                rows.append({"state": st, "region": reg, "municipality": m})
    return pd.DataFrame(rows)

def seasonal_sine(day_of_year, peak_day, amplitude, baseline):
    """Produce smooth yearly cycles (e.g., temperature/precipitation)."""
    return baseline + amplitude * np.cos(2 * np.pi * (day_of_year - peak_day) / 365.25)

def clamp(x, lo, hi):
    """Limit numeric arrays between lower and upper bounds."""
    return np.minimum(np.maximum(x, lo), hi)

# Regional climate modifiers (e.g., wetter coast, warmer inland)
REGION_CLIMATE = {
    "Great Southern": dict(temp_offset=-1.0, precip_mult=1.1, ndvi_peak_day=250),
    "Wheatbelt": dict(temp_offset=+1.0, precip_mult=0.9, ndvi_peak_day=240),
    "South Coast": dict(temp_offset=-1.5, precip_mult=1.2, ndvi_peak_day=255),
    "Yorke Peninsula": dict(temp_offset=0.0, precip_mult=1.0, ndvi_peak_day=245),
    "Mid North": dict(temp_offset=+0.5, precip_mult=0.95, ndvi_peak_day=240),
}

# ----------------------------
# Generate realistic daily weather data
# ----------------------------
def generate_daily_weather(geo_df, start_year=2016, end_year=2025):
    """
    Simulate 10 years of daily weather per municipality with realistic
    inter-annual anomalies (per year × region) to create variance across years.
    """
    start_date = dt.date(start_year, 1, 1)
    end_date = dt.date(end_year, 12, 31)
    dates = pd.date_range(start_date, end_date, freq="D")
    doy = dates.dayofyear.values

    # --- NEW: build a per-year × region anomaly table to amplify year-to-year variation ---
    years = np.arange(start_year, end_year + 1)
    anom_rows = []
    for reg in REGION_CLIMATE.keys():
        for y in years:
            anom_rows.append({
                "region": reg,
                "year": y,
                # precipitation anomaly multiplier (±15% typical)
                "precip_mult_year": np.clip(np.random.normal(1.0, 0.15), 0.7, 1.4),
                # temperature shift (°C, ±1.0 typical)
                "temp_shift_year": np.random.normal(0.0, 1.0),
                # NDVI baseline shift (±0.03 typical)
                "ndvi_shift_year": np.random.normal(0.0, 0.03),
            })
    year_anoms = pd.DataFrame(anom_rows)

    all_records = []
    for _, row in geo_df.iterrows():
        st, reg, muni = row["state"], row["region"], row["municipality"]
        rc = REGION_CLIMATE[reg]

        # Precompute static seasonal baselines (no per-year factor yet)
        tavg_base0 = seasonal_sine(doy, 30, 7.5, 16.0 + rc["temp_offset"])
        precip_base0 = seasonal_sine(doy, 210, 2.5, 1.2).clip(min=0.1)
        ndvi_peak_day = rc["ndvi_peak_day"]

        # For each year, apply a distinct anomaly to get cross-year variability at the same week
        for y in years:
            # indices for this year within the full date range
            mask_y = (dates.year == y)
            doy_y = doy[mask_y]

            # grab anomalies for (region, year)
            a = year_anoms[(year_anoms["region"] == reg) & (year_anoms["year"] == y)].iloc[0]

            # Temperature (°C): add year shift
            tavg_base = tavg_base0[mask_y] + a["temp_shift_year"]
            tmin = tavg_base - 5 + np.random.normal(0, 1.2, mask_y.sum())
            tmax = tavg_base + 7 + np.random.normal(0, 1.5, mask_y.sum())
            tavg = (tmin + tmax) / 2

            # Precipitation (mm): seasonal gamma with per-region & per-year multipliers
            precip = np.random.gamma(1.5, precip_base0[mask_y])
            precip *= rc["precip_mult"] * a["precip_mult_year"]
            precip = clamp(precip, 0, 40)

            # Soil moisture: smoothed function of precip
            soil_mo = np.zeros(mask_y.sum())
            for i in range(1, mask_y.sum()):
                soil_mo[i] = 0.1 * precip[i] + 0.9 * soil_mo[i - 1]
            sm = clamp(40 + soil_mo * 1.5 + np.random.normal(0, 3, mask_y.sum()), 10, 95)

            # NDVI: peaked season + year shift + humidity influence
            ndvi = (0.2
                    + 0.25 * np.exp(-((doy_y - ndvi_peak_day) ** 2) / (2 * 25**2))
                    + a["ndvi_shift_year"]
                    + 0.04 * (sm / 100)
                    + np.random.normal(0, 0.03, mask_y.sum()))
            ndvi = clamp(ndvi, 0.05, 0.95)

            all_records.append(pd.DataFrame({
                "date": dates[mask_y],
                "state": st,
                "region": reg,
                "municipality": muni,
                "precip_mm": precip,
                "tmin_c": tmin,
                "tmax_c": tmax,
                "tavg_c": tavg,
                "soil_rel_humidity_pct": sm,
                "ndvi": ndvi,
            }))

    df = pd.concat(all_records, ignore_index=True)
    df["year"] = df["date"].dt.year
    df["week"] = df["date"].dt.isocalendar().week.astype(int)
    return df


# 3️⃣ Simulate seasonal yields & weekly features

In [91]:
# ----------------------------
# 3️⃣ Simulate seasonal yields & weekly features
# ----------------------------

# Define crops and coefficients for synthetic yield generation
CROPS = {
    "faba_bean": dict(base=2.2, precip_beta=0.0022, temp_beta=-0.03, ndvi_beta=2.8, moist_beta=0.01),
    "chickpea":  dict(base=1.6, precip_beta=0.0016, temp_beta=-0.02, ndvi_beta=2.1, moist_beta=0.008),
    "lentil":    dict(base=1.8, precip_beta=0.0019, temp_beta=-0.022, ndvi_beta=2.3, moist_beta=0.009),
}

# Typical growing season (weeks)
SEASON_WEEKS = list(range(18, 49))

def build_seasonal_summaries(daily):
    """Aggregate daily data by season to produce one row per municipality-year."""
    return (daily[daily["week"].isin(SEASON_WEEKS)]
            .groupby(["state", "region", "municipality", "year"])
            .agg(
                precip_season=("precip_mm", "sum"),
                tavg_season=("tavg_c", "mean"),
                soil_moist_season=("soil_rel_humidity_pct", "mean"),
                ndvi_season=("ndvi", "mean"),
            )
            .reset_index())

def synthesize_yields(seasonal):
    """
    Simulate crop yields as a function of seasonal conditions,
    with NO direct precipitation effect (as requested).
    We keep temperature, NDVI and soil moisture; increase NDVI/moisture
    sensitivity a bit and keep noise modest for clearer single-variable signal.
    """
    rows = []
    for _, r in seasonal.iterrows():
        for crop, pars in CROPS.items():
            base = pars["base"]

            # y = base + (precip term) + (temp) + (ndvi) + (moist)
            y = (base
                 + (-0.028) * (r["tavg_season"] - 22)       # temp effect
                 + 3.6 * (r["ndvi_season"] - 0.35)          # ↑ NDVI sensitivity
                 + 0.014 * (r["soil_moist_season"] - 45))   # ↑ moisture sensitivity

            # Modest noise so R² reflects true signal but isn’t perfect
            y += np.random.normal(0, 0.12)

            # Crop-specific plausible ranges
            lo_hi = {"faba_bean": (1.2, 4.2), "chickpea": (0.8, 3.2), "lentil": (0.9, 3.2)}[crop]
            y = float(clamp(y, lo_hi[0], lo_hi[1]))

            rows.append(dict(
                state=r["state"], region=r["region"], municipality=r["municipality"],
                year=int(r["year"]), crop=crop, yield_t_ha=y
            ))
    return pd.DataFrame(rows)


# ----------------------------
# Weekly features for model training
# ----------------------------
def build_weekly_features(df_daily, feature_col, agg_level="municipality"):
    """Aggregate daily variable to weekly-to-date and attach 'unit' (geo name)."""
    if agg_level == "municipality":
        group = ["state", "region", "municipality", "year", "week"]
        geo_year = ["state", "region", "municipality", "year"]
        unit_col = "municipality"  # 👈 this will be used in dashboard filtering
    elif agg_level == "region":
        group = ["state", "region", "year", "week"]
        geo_year = ["state", "region", "year"]
        unit_col = "region"
    elif agg_level == "state":
        group = ["state", "year", "week"]
        geo_year = ["state", "year"]
        unit_col = "state"

    df = df_daily[df_daily["week"].isin(SEASON_WEEKS)].copy()
    weekly = df.groupby(group, as_index=False).agg({
        feature_col: ("sum" if "precip" in feature_col else "mean")
    })
    weekly = weekly.sort_values(geo_year + ["week"]).reset_index(drop=True)

    # Compute cumulative or running mean features
    def agg_to_date(x):
        return x.cumsum() if "precip" in feature_col else x.expanding().mean()
    weekly[f"{feature_col}_to_date"] = weekly.groupby(geo_year)[feature_col].transform(agg_to_date)

    # Add the 'unit' field for the dashboard to filter by region/state/municipality
    weekly["unit"] = weekly[unit_col]
    return weekly[geo_year + ["week", f"{feature_col}_to_date", "unit"]]


# 4️⃣ Modeling and dashboard setup

In [94]:
# Run 4️⃣
# 4️⃣ Modeling and dashboard setup (per-unit metrics, no squared=)
# ----------------------------

FEATURES = ["precip_mm", "tmin_c", "tmax_c", "tavg_c", "soil_rel_humidity_pct", "ndvi"]
TRAIN_YEARS = list(range(2016, 2021))
VAL_YEARS   = list(range(2021, 2024))
TEST_YEARS  = list(range(2024, 2026))

def aggregate_yields_with_unit(df_yields, agg_level="municipality"):
    """Aggregate yields by chosen level and attach a 'unit' name."""
    if agg_level == "municipality":
        keys, unit_col = ["state","region","municipality","year","crop"], "municipality"
    elif agg_level == "region":
        keys, unit_col = ["state","region","year","crop"], "region"
    else:
        keys, unit_col = ["state","year","crop"], "state"
    y = df_yields.groupby(keys, as_index=False)["yield_t_ha"].mean()
    y["unit"] = y[unit_col]
    return y

def join_keys_for(agg_level):
    """Utility to define join keys per level."""
    if agg_level == "municipality": return ["state", "region", "municipality", "year"]
    elif agg_level == "region": return ["state", "region", "year"]
    else: return ["state", "year"]

def fit_models(weather_daily, yields_yearly):
    """
    Train linear regressions at the finest requested granularity:
      - one model per (crop, feature, aggregation, unit, week).
    Each model is trained on that unit's TRAIN_YEARS only, then evaluated on that unit's VAL/TEST years.
    """
    metrics_rows, forecast_rows = [], []

    for agg_level in ["municipality", "region", "state"]:
        # Yields aggregated to chosen level, preserving 'unit'
        Y_all = aggregate_yields_with_unit(yields_yearly, agg_level)

        for feat in FEATURES:
            # Weekly-to-date feature, preserving 'unit'
            X_all = build_weekly_features(weather_daily, feat, agg_level)

            for crop in CROPS.keys():
                # Merge yields (per unit) with features (per unit-week)
                keys = join_keys_for(agg_level)
                df = (
                    Y_all[Y_all["crop"] == crop]
                    .merge(X_all, on=keys + ["unit"], how="inner")
                )

                # --- Train a separate model for EACH unit ---
                for unit_value, df_unit in df.groupby("unit", sort=False):

                    # Iterate across weeks in the season (each week = different cumulative feature)
                    for w in SEASON_WEEKS:
                        dfw = df_unit[df_unit["week"] == w].copy()
                        if dfw.empty:
                            continue

                        # Split by year for THIS unit
                        tr = dfw[dfw["year"].isin(TRAIN_YEARS)]
                        va = dfw[dfw["year"].isin(VAL_YEARS)]
                        te = dfw[dfw["year"].isin(TEST_YEARS)]

                        # Need enough samples to fit a 1-parameter linear model
                        if len(tr) < 3:  # 3+ years recommended for stability
                            # Still record rows with NaN/0 metrics so lines can render if desired
                            def _mk_zero(set_name):
                                metrics_rows.append(dict(
                                    crop=crop, variable=feat, aggregation=agg_level, unit=unit_value,
                                    week=int(w), set=set_name, r2=0.0, r2_adj=0.0,
                                    rmse=np.nan, mean_pred_yield_t_ha=np.nan
                                ))
                            for set_name in ["train", "val", "test"]:
                                _mk_zero(set_name)
                            forecast_rows.append(dict(
                                crop=crop, variable=feat, aggregation=agg_level, unit=unit_value,
                                week=int(w), mean_pred_yield_test=np.nan, mean_pred_yield_val=np.nan
                            ))
                            continue

                        # Fit the model on THIS unit's training rows
                        model = LinearRegression().fit(tr[[f"{feat}_to_date"]], tr["yield_t_ha"])

                        # Robust evaluation helper (never returns NaN R²)
                        def eval_set(sub):
                            if sub.empty:
                                return dict(r2=0.0, r2_adj=0.0, rmse=np.nan, mean_pred=np.nan)
                            yp = model.predict(sub[[f"{feat}_to_date"]])
                            y_true = sub["yield_t_ha"].values

                            # Guard: constant target or too few samples → R² undefined → force 0.0
                            if len(y_true) < 2 or np.allclose(np.var(y_true), 0.0):
                                r2 = 0.0
                                r2_adj = 0.0
                            else:
                                r2 = r2_score(y_true, yp)

                                # Clamp negative R² values to 0
                                if np.isnan(r2) or r2 < 0:
                                    r2 = 0.0

                                # Adjusted R² (also clamped to ≥ 0)
                                n = len(y_true); p = 1
                                if n > p + 1:
                                    r2_adj = 1 - (1 - r2) * (n - 1) / (n - p - 1)
                                else:
                                    r2_adj = r2
                                if np.isnan(r2_adj) or r2_adj < 0:
                                    r2_adj = 0.0

                            mse = mean_squared_error(y_true, yp)   # <- no squared= param
                            rmse = np.sqrt(mse)
                            return dict(r2=float(r2), r2_adj=float(r2_adj), rmse=float(rmse), mean_pred=float(np.nanmean(yp)))

                        # Evaluate for this unit
                        tr_m, va_m, te_m = eval_set(tr), eval_set(va), eval_set(te)

                        # Record metrics rows per set
                        for set_name, m in [("train", tr_m), ("val", va_m), ("test", te_m)]:
                            metrics_rows.append(dict(
                                crop=crop, variable=feat, aggregation=agg_level, unit=unit_value,
                                week=int(w), set=set_name,
                                r2=m["r2"], r2_adj=m["r2_adj"], rmse=m["rmse"],
                                mean_pred_yield_t_ha=m["mean_pred"]
                            ))

                        # Record forecasts for plotting
                        forecast_rows.append(dict(
                            crop=crop, variable=feat, aggregation=agg_level, unit=unit_value,
                            week=int(w),
                            mean_pred_yield_test=te_m["mean_pred"],
                            mean_pred_yield_val=va_m["mean_pred"]
                        ))

    # Assemble outputs
    metrics_df = pd.DataFrame(metrics_rows)
    forecast_df = pd.DataFrame(forecast_rows).drop_duplicates(
        subset=["crop","variable","aggregation","unit","week"]
    )

    plot_df = metrics_df.merge(
        forecast_df, on=["crop","variable","aggregation","unit","week"], how="left"
    )
    return metrics_df, plot_df


# 5️⃣ Plotly/Dash dashboard (two independent TEST panels) + open in browser

In [ ]:
# Run 5️⃣
# 5️⃣ Plotly/Dash dashboard (two TEST panels) + launcher using app.run(...)
# ----------------------------
def build_dashboard(plot_df):
    """
    Two-panel dashboard where BOTH plots use the TEST set.
    Each panel has its OWN controls (Crop, Variable, Aggregation, Unit).
    No CSS injection; dropdowns keep simple inline 'color: black' for control text.
    """
    import dash
    from dash import dcc, html, Input, Output
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    import pandas as pd

    feature_cols = ["precip_mm", "tmin_c", "tmax_c", "tavg_c", "soil_rel_humidity_pct", "ndvi"]
    aggs_available = ["municipality", "region", "state"]
    crops = sorted(plot_df['crop'].unique().tolist())
    has_unit_col = "unit" in plot_df.columns

    def unit_options_for(agg: str):
        """Return list of available units for a given aggregation level."""
        if not has_unit_col:
            return ["All"]
        opts = (plot_df.loc[plot_df["aggregation"] == agg, "unit"]
                        .dropna().drop_duplicates().sort_values().tolist())
        return opts if opts else ["All"]

    app = dash.Dash(__name__)

    def test_series_for(crop, var_sel, agg_sel, unit_value):
        sel = (
            (plot_df["crop"] == crop) &
            (plot_df["variable"] == var_sel) &
            (plot_df["aggregation"] == agg_sel) &
            (plot_df["set"] == "test")
        )
        df = plot_df.loc[sel].copy()
        if has_unit_col and pd.notna(unit_value) and unit_value != "All":
            df = df[df["unit"] == unit_value]
        df = df.sort_values("week")

        # Coerce numeric & fill NaNs so the line always draws
        w = df["week"].astype(int)
        y_pred = pd.to_numeric(df["mean_pred_yield_test"], errors="coerce").astype(float)
        r2_adj = pd.to_numeric(df["r2_adj"], errors="coerce").astype(float).fillna(0.0)

        return w, y_pred, r2_adj

    app.layout = html.Div([
        html.H2("Crop Yield Forecast vs Model Performance (Weekly) — TEST only"),

        # ====== Panel A (Top) ======
        html.Div([
            html.Div([
                html.Strong("Top plot (TEST) controls"),

                html.Label("Crop"),
                dcc.Dropdown(
                    options=[{"label": c.replace("_"," ").title(), "value": c} for c in crops],
                    value=crops[0], id="crop-dd-a", clearable=False,
                    style={"width": "220px", "color": "black"},
                ),

                html.Label("Variable"),
                dcc.Dropdown(
                    options=[{"label": v, "value": v} for v in feature_cols],
                    value="ndvi", id="var-dd-a", clearable=False,
                    style={"width": "220px", "color": "black"},
                ),

                html.Label("Aggregation"),
                dcc.Dropdown(
                    options=[{"label": a.title(), "value": a} for a in aggs_available],
                    value="municipality", id="agg-dd-a", clearable=False,
                    style={"width": "220px", "color": "black"},
                ),

                html.Label("Unit"),
                dcc.Dropdown(
                    options=[{"label": u, "value": u} for u in unit_options_for("municipality")],
                    value=unit_options_for("municipality")[0], id="unit-dd-a", clearable=False,
                    style={"width": "260px", "color": "black"},
                ),
            ], style={"display":"flex","gap":"16px","alignItems":"center","flexWrap":"wrap","marginBottom":"8px"}),

            dcc.Graph(id="weekly-fig-a", style={"height":"400px"}),
        ], style={"marginBottom": "24px"}),

        # ====== Panel B (Bottom) ======
        html.Div([
            html.Div([
                html.Strong("Bottom plot (TEST) controls"),

                html.Label("Crop"),
                dcc.Dropdown(
                    options=[{"label": c.replace("_"," ").title(), "value": c} for c in crops],
                    value=crops[0], id="crop-dd-b", clearable=False,
                    style={"width": "220px", "color": "black"},
                ),

                html.Label("Variable"),
                dcc.Dropdown(
                    options=[{"label": v, "value": v} for v in feature_cols],
                    value="ndvi", id="var-dd-b", clearable=False,
                    style={"width": "220px", "color": "black"},
                ),

                html.Label("Aggregation"),
                dcc.Dropdown(
                    options=[{"label": a.title(), "value": a} for a in aggs_available],
                    value="region", id="agg-dd-b", clearable=False,
                    style={"width": "220px", "color": "black"},
                ),

                html.Label("Unit"),
                dcc.Dropdown(
                    options=[{"label": u, "value": u} for u in unit_options_for("region")],
                    value=unit_options_for("region")[0], id="unit-dd-b", clearable=False,
                    style={"width": "260px", "color": "black"},
                ),
            ], style={"display":"flex","gap":"16px","alignItems":"center","flexWrap":"wrap","marginBottom":"8px"}),

            dcc.Graph(id="weekly-fig-b", style={"height":"400px"}),
        ]),
    ])

    # Sync Unit options with Aggregation (Panel A)
    @app.callback(
        Output("unit-dd-a", "options"), Output("unit-dd-a", "value"),
        Input("agg-dd-a", "value"),
    )
    def sync_units_a(agg_value):
        opts = unit_options_for(agg_value)
        return ([{"label": u, "value": u} for u in opts], opts[0])

    # Sync Unit options with Aggregation (Panel B)
    @app.callback(
        Output("unit-dd-b", "options"), Output("unit-dd-b", "value"),
        Input("agg-dd-b", "value"),
    )
    def sync_units_b(agg_value):
        opts = unit_options_for(agg_value)
        return ([{"label": u, "value": u} for u in opts], opts[0])

    # Update Top plot (TEST)
    @app.callback(
        Output("weekly-fig-a", "figure"),
        Input("crop-dd-a", "value"), Input("var-dd-a", "value"),
        Input("agg-dd-a", "value"), Input("unit-dd-a", "value"),
    )
    def update_fig_a(crop_a, var_a, agg_a, unit_a):
        w, y_pred, r2a = test_series_for(crop_a, var_a, agg_a, unit_a)
        fig = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])
        fig.add_trace(go.Scatter(x=w, y=y_pred, mode="lines+markers", name="Predicted yield (t/ha) — Test"), secondary_y=False)
        fig.add_trace(go.Scatter(x=w, y=r2a, mode="lines+markers", name="Adj. R² — Test"), secondary_y=True)
        fig.update_layout(title=f"TEST — {crop_a.replace('_',' ').title()} | {var_a} | {agg_a.title()} [{unit_a}]",
                          height=400, legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="center", x=0.5))
        fig.update_xaxes(title_text="Week (18–48)")
        fig.update_yaxes(title_text="Predicted Yield (t/ha)", secondary_y=False)
        fig.update_yaxes(title_text="Adjusted R²", secondary_y=True)
        return fig

    # Update Bottom plot (TEST)
    @app.callback(
        Output("weekly-fig-b", "figure"),
        Input("crop-dd-b", "value"), Input("var-dd-b", "value"),
        Input("agg-dd-b", "value"), Input("unit-dd-b", "value"),
    )
    def update_fig_b(crop_b, var_b, agg_b, unit_b):
        w, y_pred, r2b = test_series_for(crop_b, var_b, agg_b, unit_b)
        fig = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])
        fig.add_trace(go.Scatter(x=w, y=y_pred, mode="lines+markers", name="Predicted yield (t/ha) — Test"), secondary_y=False)
        fig.add_trace(go.Scatter(x=w, y=r2b, mode="lines+markers", name="Adj. R² — Test"), secondary_y=True)
        fig.update_layout(title=f"TEST — {crop_b.replace('_',' ').title()} | {var_b} | {agg_b.title()} [{unit_b}]",
                          height=400, legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="center", x=0.5))
        fig.update_xaxes(title_text="Week (18–48)")
        fig.update_yaxes(title_text="Predicted Yield (t/ha)", secondary_y=False)
        fig.update_yaxes(title_text="Adjusted R²", secondary_y=True)
        return fig

    return app

# 👉 Launcher: open in a browser and run using app.run(...)
def launch_dashboard_in_browser(plot_df, host="127.0.0.1", port=8050):
    import webbrowser, threading
    app = build_dashboard(plot_df)
    url = f"http://{host}:{port}"
    threading.Timer(1.0, lambda: webbrowser.open(url)).start()   # open browser after a short delay
    app.run(host=host, port=port, debug=False)                   # NOTE: using app.run(...), not run_server


# 6️⃣ Pipeline runner (Jupyter): generate data, train models, save CSVs, launch dashboard

In [95]:
# Run 6️⃣
# 6️⃣ Pipeline runner (Jupyter): generate data, train models, save CSVs, launch dashboard
# ----------------------------
import os  # ensure available for os.path.join

# 1) Build the geography table (25 municipalities across 5 regions & 2 states)
geo_df = build_geo()

# 2) Generate daily weather & NDVI for 2016–2025
weather_daily = generate_daily_weather(geo_df, 2016, 2025)

# 3) Aggregate to seasonal summaries and synthesize yearly crop yields
seasonal = build_seasonal_summaries(weather_daily)
yields_yearly = synthesize_yields(seasonal)

# 4) Save base datasets — 👇 all paths now built using os.path.join
weather_csv = os.path.join(".", OUTDIR, "au_crop_weather_daily.csv")
yields_csv  = os.path.join(".", OUTDIR, "au_crop_yields_yearly.csv")
weather_daily.to_csv(weather_csv, index=False)
yields_yearly.to_csv(yields_csv, index=False)
print(f"✓ Saved: {weather_csv}")
print(f"✓ Saved: {yields_csv}")

# 5) Fit weekly simple linear models (single predictor) and build plot_df with 'unit'
metrics_df, plot_df = fit_models(weather_daily, yields_yearly)

# 6) Save model outputs (full metrics + plotting dataset)
metrics_csv = os.path.join(".", OUTDIR, "model_weekly_metrics_allsets.csv")
plot_csv    = os.path.join(".", OUTDIR, "model_weekly_metrics_and_forecasts.csv")
metrics_df.to_csv(metrics_csv, index=False)
plot_df.to_csv(plot_csv, index=False)
print(f"✓ Saved: {metrics_csv}")
print(f"✓ Saved: {plot_csv}")

# 7) Launch dashboard (inline in Jupyter or local server)
# try:
#     from jupyter_dash import JupyterDash
#     app = build_dashboard(plot_df)
#     app.__class__ = JupyterDash
#     display(app.run(mode="inline", height=900, port=8050))
# except Exception as e:
#     print("Inline mode not available; starting a local server instead...")
#     app = build_dashboard(plot_df)
#     app.run(debug=False, port=8050)


✓ Saved: ./output/au_crop_weather_daily.csv
✓ Saved: ./output/au_crop_yields_yearly.csv
✓ Saved: ./output/model_weekly_metrics_allsets.csv
✓ Saved: ./output/model_weekly_metrics_and_forecasts.csv


In [96]:
# Launch dashboard
launch_dashboard_in_browser(plot_df)  # opens http://127.0.0.1:8050 in your browser
